In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# DATA OVERVIEW

In [2]:
import pandas as pd 
data=pd.read_csv("accepted_2007_to_2018Q4.csv")
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data.shape

(2260701, 151)

Lending club data consists of 2,260,701 rows and 151 columns. The source of the data is from the Lending Club Website or Kaggle competitions have the latest updated datasets. The target column was identified as loan_status. For this analysis only Fully Paid Loans and Charged off/ Defaulted Loans have been taken into consideration, which reduced the data set to 1,345,383 rows and 151 columns.

# TARGET VARIABLE

For now we will drop all the categories except 'Fully Paid', 'Default' and 'Charged off'. we will also merge 'Charged off' and 'Default' together, because anyone who fell into 'Charged off' category defaulted their loan. A charge-off is a debt that a creditor has given up trying to collect on after the borrower has missed payments for several months. This categorization helps break the data into a binary column.

Now encoding the two categories listed above as 0 or 1. This will help in predicting whether a person defaulted their loan or not. 0 means the borrower has defaulted and 1 means the borrower paid off the loan.

In [4]:
data['loan_status'].value_counts()

Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: loan_status, dtype: int64

In [5]:
data.shape

(2260701, 151)

In [6]:
data=data[data['loan_status']!='Late (16-30 days)']
data=data[data['loan_status']!='Late (31-120 days)']
data=data[data['loan_status']!='In Grace Period']
data=data[data['loan_status']!='Current']
data=data[data['loan_status']!='Does not meet the credit policy. Status:Fully Paid']
data=data[data['loan_status']!='Does not meet the credit policy. Status:Charged Off']

In [7]:
data.shape

(1345383, 151)

In [8]:
data['target']=data['loan_status'].replace({'Fully Paid':0,'Charged Off':1,'Default':1})

In [9]:
data['target'].value_counts(normalize=True)

0.0    0.80035
1.0    0.19965
Name: target, dtype: float64

In [10]:
data['target'].value_counts()

0.0    1076751
1.0     268599
Name: target, dtype: int64

# DATA CLEANING

Data leakage is when information from outside the training dataset is used to create the model. It’s hard because we cannot evaluate the model on something we don’t have.
Therefore, we must estimate the performance of the model on unseen data by training it on only some of the data we have and evaluating it on the rest of the data.
In our case, few features were recorded after the loan commencement. Such features are not available to the user in reality when he is predicting default. Thus, these features must be eliminated and must not be included as part of the prediction model. With this, the columns were reduced from 151 to 88.

In [11]:
leak_cols = ['acc_now_delinq', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 
             'collection_recovery_fee', 'collections_12_mths_ex_med', 'debt_settlement_flag', 'delinq_2yrs', 'delinq_amnt', 
             'disbursement_method', 'funded_amnt', 'hardship_flag', 'inq_last_6mths', 'last_credit_pull_d', 
             'initial_list_status', 'mo_sin_old_rev_tl_op', 'mo_sin_old_il_acct','last_pymnt_amnt', 'last_pymnt_d', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 
             'mths_since_recent_bc', 'mths_since_recent_inq', 'num_accts_ever_120_pd',
             'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 
             'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m',  
             'out_prncp', 'out_prncp_inv', 'pct_tl_nvr_dlq',     'percent_bc_gt_75', 'pymnt_plan', 'recoveries', 
             'tax_liens', 'tot_coll_amt', 'tot_cur_bal', 'last_fico_range_high','last_fico_range_low','url','id','title','addr_state',
             'tot_hi_cred_lim','total_bal_ex_mort', 'total_bc_limit',
             'total_il_high_credit_limit', 'total_pymnt', 'total_pymnt_inv', 
             'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 
             'total_rev_hi_lim']

data.drop(leak_cols,axis=1,inplace=True)
data.shape

(1345383, 88)

# MISSING DATA

We kept a threshold value of 70% for missing data, which meant removing columns with more than 70% null values.Since majority of the features had more than 90% missing dataimputations wouldn't be very helpful.
This reduced the number of features from 88 to 46

In [12]:
# Columns with  null values more than 70%

s=data.isnull().sum()*100/len(data)
x=s[s>70]

In [13]:
x=pd.DataFrame(x)
x.reset_index(inplace=True)
l=list(x.iloc[:,0])
print(l)

['member_id', 'desc', 'mths_since_last_record', 'next_pymnt_d', 'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'mths_since_recent_bc_dlq', 'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog', 'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term', 'hardship_amount', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_length', 'hardship_dpd', 'hardship_loan_status', 'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount', 'hardship_last_payment_amount', 'debt_settlement_flag_date', 'settlement_status', 'settlement_date', 'settlement_amount', 'settlement_percentage', 'settlement_term']

In [14]:
data.drop(l,axis=1,inplace=True)
data.shape

(1345383, 46)

## CATEGORICAL FEATURES

In [15]:
#remove columns that have only 1 feature
#get unique values per feature
nuni_val = data.apply(pd.Series.nunique)

#note columns that have single value
single_cat_cols = nuni_val[nuni_val == 1].index.tolist()
print(single_cat_cols)

['policy_code']


In [16]:
data = data.drop(columns=single_cat_cols)

In [17]:
data.shape

(1345383, 45)

In [18]:
#we dropped these columns as they have no direct relationship with the problem statement

data.drop(['funded_amnt_inv','mths_since_rcnt_il'],axis=1,inplace=True)
data.shape

(1345383, 43)

In [19]:
data.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'purpose', 'zip_code',
       'dti', 'earliest_cr_line', 'fico_range_low', 'fico_range_high',
       'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'application_type', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi',
       'total_cu_tl', 'inq_last_12m', 'mort_acc',
       'mths_since_recent_revol_delinq', 'pub_rec_bankruptcies', 'target'],
      dtype='object')

# NUMERICAL DATA

In [ ]:
#initially there are only 30 numerical columns after working on categorical columns we have 37 numerical columns
#

In [35]:
#Numerical columns

data_num=data.select_dtypes(np.number)
len(data_num.columns)

37

# CATEGORICAL DATA

We will further drop 'emp_title' as it 27900 unique categories

'sub_grade' will be dropped as it has 35 unique categories

'loan_status' will be dropped as we have converted and renamed it into 'target'

'zip_code' will be dropped as they are not a good representation of real human behavior, and when used in data analysis, often mask real, underlying insights, and may ultimately lead to bad outcomes.

'application_type' can be dropped as it is a severely imbalanced feature(for now converted to numerical using get_dummies)

'issue_d' and 'earliest_cr_line' will be converted to numerical as they are wrongly classified as categorical


'emp_length','purpose','home_ownership','grade','term','verification_status', will be converted to numerical



In [22]:
#Initially 13 but we will make it 0

In [36]:
#Categorical columns

data_cat=data.select_dtypes(object)
len(data_cat.columns)

0

In [24]:
for i in data_cat:
    print()
    print(i,' ',data_cat[i].nunique())


term   2

grade   7

sub_grade   35

emp_title   378358

emp_length   11

home_ownership   6

verification_status   3

issue_d   139

loan_status   3

purpose   14

zip_code   943

earliest_cr_line   739

application_type   2


In [25]:
data.application_type.value_counts(normalize=True)

Individual    0.980818
Joint App     0.019182
Name: application_type, dtype: float64

In [26]:
data.drop(['emp_title','zip_code','loan_status','sub_grade'],axis=1,inplace=True)
data.shape

(1345383, 39)

In [27]:
# Changing employee length to numerical.

data['emp_length'].fillna("< 1 year",inplace= True)
dict_emp_length = {'10+ years':10, '6 years':6, '4 years':4, '< 1 year':0.5, '2 years':2,'9 years':9, '5 years':5, '3 years':3, '7 years':7, '1 year':1,'8 years':8}
data['emp_length'].replace(dict_emp_length, inplace=True)

In [28]:
# Changing purpose into numerical using labelencoder
from sklearn.preprocessing import LabelEncoder

col=['purpose']
enc= LabelEncoder()
data.loc[:,col]= data.loc[:,col].apply(enc.fit_transform)

In [29]:
# Changing Grade to numerical.

grade_to_int = dict(zip(['A', 'B', 'C', 'D', 'E', 'F', 'G'], np.arange(7, 0, -1)))
data['grade'].replace(grade_to_int,inplace=True)

In [30]:
# Changing Home ownership to numerical.

home_to_int = {'MORTGAGE': 4, 'RENT': 3, 'OWN': 5,'ANY': 2,'OTHER': 1, 'NONE':0 }
data['home_ownership'].replace(home_to_int,inplace=True)

In [31]:
# Changing Term to numerical.

term_to_int = {' 36 months': 36,' 60 months': 60}
data['term'].replace(term_to_int,inplace=True)

In [32]:
# Changing Verification Status to numerical.

ver_stat_to_int = {'Source Verified':2,'Verified': 1,'Not Verified': 0} 
data['verification_status'].replace(ver_stat_to_int,inplace=True)

In [33]:
# Coverting Earliest Credit Line and Issue Date to numerical

import datetime as dt
data['earliest_cr_line']=pd.to_datetime(data.earliest_cr_line, format='%b-%Y')
data['issue_d']=pd.to_datetime(data.issue_d, format='%b-%Y')



TypeError: cannot astype a datetimelike from [datetime64[ns]] to [float64]

In [ ]:
data['earliest_cr_line']=data['earliest_cr_line'].astype(float)
data['issue_d']=data['issue_d'].astype(float)

In [ ]:
data['earliest_cr_line'] = data["earliest_cr_line"].dt.strftime("%m")
data['issue_d'] = data["issue_d"].dt.strftime("%m")

In [34]:
data=pd.get_dummies(data,columns=['application_type'],drop_first=True)

# MISSING VALUES AND TREATMENT

In [37]:
data.isnull().sum().sort_values(ascending=False)

mths_since_recent_revol_delinq    895407
il_util                           880336
all_util                          807800
inq_last_12m                      807748
total_cu_tl                       807748
open_acc_6m                       807748
max_bal_bc                        807747
open_rv_24m                       807747
open_il_12m                       807747
open_il_24m                       807747
total_bal_il                      807747
open_rv_12m                       807747
open_act_il                       807747
inq_fi                            807747
mths_since_last_delinq            678794
mort_acc                           47314
revol_util                           890
pub_rec_bankruptcies                 730
dti                                  407
target                                33
loan_amnt                             33
total_acc                             33
term                                  33
int_rate                              33
installment     

In [44]:
# Checking for null values
# Percentage of null values
a = (data.isnull().sum()/data.shape[0])*100
b = a[a>0.05]
b = pd.DataFrame(b,columns=['Percentage of null values'])
b.sort_values(by=['Percentage of null values'],ascending=False)

,Percentage of null values
mort_acc,3.516768
revol_util,0.066152
pub_rec_bankruptcies,0.054260


Since our dataset is huge and we have more than 50% null values in many columns we are keeping a threshold of 50 and dropping them because imputing them might lead to errors during model building.

In [39]:
data.shape       #before dropping columns with null values more than 50%

(1345383, 39)

In [40]:
# Columns with  null values more than 50%

s=data.isnull().sum()*100/len(data)
x=s[s>50]

In [41]:
x=pd.DataFrame(x)
x.reset_index(inplace=True)
l=list(x.iloc[:,0])
print(l)

['mths_since_last_delinq', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'mths_since_recent_revol_delinq']


In [42]:
data.drop(l,axis=1,inplace=True)
data.shape

(1345383, 24)

In [45]:
# imputing mort_acc, revol_util and pub_rec_bankruptcies with median

data['mort_acc']=data['mort_acc'].fillna(data['mort_acc'].median())
data['revol_util']=data['revol_util'].fillna(data['revol_util'].median())
data['pub_rec_bankruptcies']=data['pub_rec_bankruptcies'].fillna(data['pub_rec_bankruptcies'].median())

In [46]:
data.dropna(inplace=True)
data.shape

# We can see that the missing values belong to same rows and very insignificant reduce of 407 rows in the entire dataset.

(1344976, 24)

# Further Simplifying based on Domain

In [47]:
data.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'purpose', 'dti', 'earliest_cr_line', 'fico_range_low',
       'fico_range_high', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'mort_acc', 'pub_rec_bankruptcies', 'target',
       'application_type_Joint App'],
      dtype='object')

In [60]:
#To find the credit history or duration of a credit line we use earliest credit line open date and issue date of loan
# since we found the realtion between the 2 features we can drop the parent

data['credit_history']=data['issue_d']-data['earliest_cr_line']

data.drop(['issue_d','earliest_cr_line'],axis=1,inplace=True)

In [49]:
#Creditline ratio is the ratio between open credit lines and total credit lines.
# So since creditline ratio has both the features we can drop the parents

data['creditline_ratio']=data['open_acc']/data['total_acc']
data.drop(['open_acc','total_acc'],axis=1,inplace=True)

In [50]:
#Using loan_amount interest_rate and term we calculate emi of each record

p=data['loan_amnt']
r=data['int_rate']
t=data['term']


data['emi']=(p*r*((1+r)**t))/((1+r)**(t-1))

In [56]:
# Balance income is the money used by borrower for other expenses other than the loan
# A person with a high balance income is more likely to repay a loan

data['balance_income']=data['loan_amnt']/data['annual_inc']

In [ ]:
term_to_int = {36: 1,60: 2}
data['term'].replace(term_to_int,inplace=True)

In [57]:
data.shape

(1344976, 25)

In [52]:
#Using fico_high and fico_low we calculate the average fico score.
#We can then take avg_fico for making predictions and drop the original columns

a=data['fico_range_high']
b=data['fico_range_low']

data['avg_fico']=(a+b)/2

In [53]:
data.drop(['fico_range_high','fico_range_low'],axis=1,inplace=True)

# COLUMNS BEING CONSIDERED FOR ANALYSIS

In [66]:
data.shape

(1344976, 23)

In [62]:
data.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'purpose', 'dti',
       'pub_rec', 'revol_bal', 'revol_util', 'mort_acc',
       'pub_rec_bankruptcies', 'target', 'application_type_Joint App',
       'credit_history', 'creditline_ratio', 'emi', 'avg_fico',
       'balance_income'],
      dtype='object')

In [69]:
data.to_csv('final_data_capstone.csv',index=False)

REFERENCE
(reference : https://towardsdatascience.com/stop-using-zip-codes-for-geospatial-analysis-ceacb6e80c38 )